# Импорты

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from glob import glob
#from progiter import ProgIter
import joblib

from sklearn import metrics

import torch
from torch.utils.data import TensorDataset, DataLoader

from transformers import AutoTokenizer

import warnings

warnings.filterwarnings('ignore')

# Создание датасета

In [36]:
def make_dataset(texts, labels):
    '''
    Токенизация текстов и сопоставление токенов с идентификаторами
    соответствующих им слов. Формирование PyTorch датасета
    '''
    input_ids = []        # Список для токенизированных текстов
    attention_masks = []  # Список для масок механизма внимания
    
    # Цикл проходится и токенизирует каждый текст
    for seq_to_token in texts:
        encoded_dict = tokenizer.encode_plus(
            seq_to_token,                # Последовательность для токенизации
            add_special_tokens=True,     # Добавить специальные токены в начало и в конец посл-ти
            max_length=338,              # Максимальная длина последовательности
            padding='max_length',        # Токен для заполнения до максимальной длины
            return_attention_mask=True,  # Маска механизма внимания для указания на паддинги
            return_tensors = 'pt',       # Возвращать pytorch-тензоры
            truncation=True              # Обрезать последовательность до максимальной длины
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    
    # Конкатенация входных данных в тензоры
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    # Преобразование таргетов в тензоры
    labels = torch.tensor(labels.values)
    # Формирование датасета
    dataset = TensorDataset(input_ids, attention_masks, labels)

    return dataset

# Алгоритмы обучения и проверки

In [3]:
def validate():
    '''
    Валидация обученной модели на тестовой выборке
    '''
    print(f'Validation')
    model.eval()             
    fin_targets = []         # Список для всех таргетов валидационной выборки
    fin_outputs = []         # Список для всех предиктов модели на валидационной выборки
    total_test_loss = 0.0    # Функция потерь на валидации
    
    with torch.no_grad():
        # Без подсчета градиентов цикл проходится по батчам
        for data in test_dataloader:
            ids = data[0].to(device, dtype=torch.long)            # Токены последовательностей из батча
            mask = data[1].to(device, dtype=torch.long)           # Маски механизма внимания последовательностей
            targets = data[2].to(device, dtype=torch.float)       # Таргеты из батча
                
            res = model(ids, attention_mask=mask, labels=targets) # В модель подаются входные тензоры и таргеты
            loss = res['loss']                                    # Вычисляется значение функции потерь
            logits = res['logits']                                # Логиты предсказаний модели
            total_test_loss += loss.item()                        # Считается функция потерь
            
            # Таргеты и выходы модели по батчу добавляются в списки. Логиты проходят через сигмоиду
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(logits).cpu().detach().numpy().tolist())
    
    fin_targets = np.array(fin_targets)
    fin_outputs = np.array(fin_outputs)
    predictions = np.zeros(fin_outputs.shape)
    predictions[np.where(fin_outputs >= 0.5)] = 1
    
    return total_test_loss / len(test_dataloader), fin_targets, predictions

# Метрики

In [5]:
def hamming_distance(y_test, y_pred):
    '''
    Расчёт доработанного расстояния Хэмминга с ориентацией на целевое значение = 1

    y_test - тестовые целевые данные

    y_pred - предсказанные целевые данные

    Вывод среднего значения расстояния Хэмминга по всем элементам 

    '''
    try:
        y_true = y_test.to_numpy()
    except:
        y_true = y_test

    if (len(y_true) != len(y_pred)):
        raise Exception('Объекты должны быть одинаковой длины')
    distance_list = []
    
    for elem in range(len(y_true)):
        if (len(y_true[elem]) != len(y_pred[elem])):
            raise Exception('Элементы должны быть одинаковой длины')

        indices_1_y_true = set(np.where(y_true[elem])[0])
        indices_1_y_pred = set(np.where(y_pred[elem])[0])
        
        if len(indices_1_y_true) == 0 and len(indices_1_y_pred) == 0:
            dist_counter = 1

        else:
            dist_counter = len(indices_1_y_true.intersection(indices_1_y_pred)) / float(len(indices_1_y_true.union(indices_1_y_pred)))

        distance_list.append(dist_counter)
    return round(np.mean(distance_list),5)

In [28]:
def estimation(targets, outputs):
    metrics_dict = {
        'Accuracy': metrics.accuracy_score(targets, outputs),
        'Hamming_distance': hamming_distance(targets, outputs),
        'F1_micro': metrics.f1_score(targets, outputs, average='micro'),
        'F1_macro': metrics.f1_score(targets, outputs, average='macro'),
        'Recall_micro': metrics.recall_score(targets, outputs, average='micro'),
        'Recall_macro': metrics.recall_score(targets, outputs, average='macro'),
        'Precision_micro': metrics.precision_score(targets, outputs, average='micro', zero_division=0.0),
        'Precision_macro': metrics.precision_score(targets, outputs, average='macro', zero_division=0.0)
    }
    return metrics_dict

# Загрузка датасета с примерами

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

df_test = pd.read_csv('D:\\netology_diplom\\final\\csv_files\\example.csv')
X_test, y_test = df_test['text_lemma'], df_test.drop('text_lemma', axis=1)
y_test.fillna(int(0), inplace=True)


cuda


# Результаты

## Расчёт метрик на тестовом датасете для каждой модели

In [8]:
import os
models = []
for file in os.listdir('D:\\netology_diplom\\final\\code_files\\models\\'):
    models.append(file)

In [9]:
models

['ai-forever-rubert-base.pt',
 'cointegrated-rubert-tiny2_14.pt',
 'FacebookAI-xlm-roberta-base_10.pt',
 'Geotrend-distilbert-base-ru-cased_12.pt',
 'model_tfidf_catboost_ngram_1_1.pkl',
 'model_tfidf_logreg_ngram_1_1.pkl',
 'model_tfidf_logreg_ngram_1_2.pkl']

### Расчёт метрик на тестовом датасете для классических моделей

In [55]:
y_pred_dict = {}
metrics_df = pd.DataFrame()
for model_name in models:
    if '.pkl' in model_name:
        print(model_name)
        row_no = len(metrics_df)
        metrics_df.loc[row_no, 'model_name'] = model_name[6:-4]
        model = joblib.load('D:\\netology_diplom\\final\\code_files\\models\\'+model_name)
        y_pred = model.predict(X_test)
        y_pred_dict[model_name[6:-4]] = y_pred
        metrics_dict = estimation(y_test.to_numpy(), y_pred)
        for metric, value in metrics_dict.items():
            metrics_df.loc[row_no, metric] = value
    else:
        pass

model_tfidf_catboost_ngram_1_1.pkl
model_tfidf_logreg_ngram_1_1.pkl
model_tfidf_logreg_ngram_1_2.pkl


### Расчёт метрик на тестовом датасете для нейросетевых моделей

In [61]:
for model_name in models:
    if '.pt' in model_name:
        print(model_name)
        row_no = len(metrics_df)
        model_name = model_name[:-3]
        metrics_df.loc[row_no, 'model_name'] = model_name
        selected_model = input()
        tokenizer = AutoTokenizer.from_pretrained(selected_model) # ai-forever/rubert-base cointegrated/rubert-tiny2 FacebookAI/xlm-roberta-base Geotrend/distilbert-base-ru-cased
        test_dataset = make_dataset(X_test, y_test)
        test_dataloader = DataLoader(test_dataset)
        model = torch.load('D:\\netology_diplom\\final\\code_files\\models\\' + model_name + '.pt')
        avg_val_loss, targets, y_pred = validate()
        y_pred_dict[model_name] = y_pred
        metrics_dict = estimation(targets, y_pred)
        for metric, value in metrics_dict.items():
            metrics_df.loc[row_no, metric] = value
    else:
        pass

ai-forever-rubert-base.pt
Validation
cointegrated-rubert-tiny2_14.pt
Validation
FacebookAI-xlm-roberta-base_10.pt
Validation
Geotrend-distilbert-base-ru-cased_12.pt
Validation


### Постобработка данных

In [ ]:
metrics_df = metrics_df.drop(labels = [3,4])
metrics_df.reset_index(drop=True,inplace = True)
df_metrics = metrics_df.copy()
metrics_df.loc[row_no, 'avg_val_loss'] = avg_val_loss
metrics_df = df_metrics

### Результаты в виде сводной таблицы

In [ ]:
metrics_df
metrics_df.to_csv('metrics_pred.csv', index=False)

,model_name,Accuracy,Hamming_distance,F1_micro,F1_macro,Recall_micro,Recall_macro,Precision_micro,Precision_macro
0,tfidf_catboost_ngram_1_1,0.424242,0.42803,0.567164,0.463884,0.431818,0.426966,0.826087,0.558989
1,tfidf_logreg_ngram_1_1,0.367424,0.47898,0.523810,0.516887,0.625000,0.617978,0.450820,0.496889
2,tfidf_logreg_ngram_1_2,0.382576,0.49463,0.552632,0.532694,0.636364,0.629213,0.488372,0.514968
3,ai-forever-rubert-base,0.560606,0.56061,0.593186,0.517918,0.560606,0.554307,0.629787,0.544868
4,cointegrated-rubert-tiny2_14,0.503788,0.50568,0.565401,0.482292,0.507576,0.501873,0.638095,0.521629
5,FacebookAI-xlm-roberta-base_10,0.545455,0.54735,0.581162,0.493810,0.549242,0.543071,0.617021,0.485326
6,Geotrend-distilbert-base-ru-cased_12,0.522727,0.52841,0.564000,0.493053,0.534091,0.528090,0.597458,0.505939


# Сохранение предсказаний по каждой модели

In [75]:
columns = df_test.columns[1:]
for k,v in y_pred_dict.items():
    print(k)
    df_pred = pd.DataFrame(data=v, columns=columns)
    df_pred.to_csv('df_pred_' + k + '.csv', index = False)

tfidf_catboost_ngram_1_1
tfidf_logreg_ngram_1_1
tfidf_logreg_ngram_1_2
ai-forever-rubert-base
cointegrated-rubert-tiny2_14
FacebookAI-xlm-roberta-base_10
Geotrend-distilbert-base-ru-cased_12


# Примеры и сравнение

In [79]:
df_ex = pd.read_csv('example.csv')
df_ex.fillna(0, inplace = True)
columns_lst = df_ex.columns[1:]
df_final_example = df_ex.drop(columns=columns_lst)

models_list_comparison = ['ai-forever-rubert-base.pt',
 'cointegrated-rubert-tiny2_14.pt',
 'FacebookAI-xlm-roberta-base_10.pt',
 'Geotrend-distilbert-base-ru-cased_12.pt',
 'model_tfidf_catboost_ngram_1_1.pkl',
 'model_tfidf_logreg_ngram_1_1.pkl',
 'model_tfidf_logreg_ngram_1_2.pkl']

for k in models_list_comparison:
    print(k[:-3])
    try:
        df_pred = pd.read_csv('df_pred_' + k[:-3] + '.csv')
    except:
        df_pred = pd.read_csv('df_pred_' + k[6:-4] + '.csv')
    indices_pred_list = []
    indices_true_list = []
    for elem in range(len(df_pred)):
        indices_pred = set(np.where(df_pred.to_numpy()[elem])[0])
        indices_true = set(np.where(df_ex[columns_lst].to_numpy()[elem])[0])
        indices_pred_list.append(indices_pred)
        indices_true_list.append(indices_true)
    df_final_example[f'id_true_{k[:-3]}'] = indices_true_list
    df_final_example[f'id_pred_{k[:-3]}'] = indices_pred_list
columns_final_ex = df_final_example.columns

ai-forever-rubert-base
cointegrated-rubert-tiny2_14
FacebookAI-xlm-roberta-base_10
Geotrend-distilbert-base-ru-cased_12
model_tfidf_catboost_ngram_1_1.
model_tfidf_logreg_ngram_1_1.
model_tfidf_logreg_ngram_1_2.


In [ ]:
def comparison_result(st_ind,end_ind,columns_lst, columns_final_ex):
    for col in range(1,15,2):
        #    print(col)
        st_col, end_col = columns_final_ex[col], columns_final_ex[col+1]
        print(st_col, end_col)
        for i in range(len(df_final_example[st_ind:end_ind].loc[:,['text_lemma',st_col,end_col]])):
            text_lemm = df_final_example[st_ind:end_ind].loc[:,['text_lemma',st_col,end_col]].iloc[i][0]
            true_tag = columns_lst[list(df_final_example[st_ind:end_ind].loc[:,['text_lemma',st_col,end_col]].iloc[i][1])[0]]
            print('model:', st_col[8:])
            print('text_lemmatize:', text_lemm)
            print('true_tag:', true_tag)
            try:
                pred_tag = list(df_final_example[st_ind:end_ind].loc[:,['text_lemma',st_col,end_col]].iloc[i][2])
                list_pred = []
                for tag in pred_tag:
                    list_pred.append(columns_lst[tag])
                print('pred_tag:',', '.join(list_pred))
            except:
                pred_tag = ''
                print('pred_tag:', pred_tag)
            print()

In [187]:
comparison_result(170,171,columns_lst, columns_final_ex)

id_true_ai-forever-rubert-base id_pred_ai-forever-rubert-base
model: ai-forever-rubert-base
text_lemmatize: президент россия владимир путин свой указ производить генерал полковник различный ведомство министерство оборона военный прокуратура соответствующий указ среда декабрь опубликовывать официальный портал правовой информация кроме генерал присваивать очередной воинский звание частность минобороны россия появляться новый генерал-полковник полный адмирал восемь генерал-лейтенант приравнивать вице-адмирал генерал-майор контр-адмирал мчс появиллось генерал-полковник четыре генерал-лейтенант семь генерал-майор свой очередь росгвардия пополняться генерал-полковник генерал-лейтенант девять генерал-майор военный прокуратура появляться генерал-лейтенант юстиция генерал-майор федеральный служба исполнение наказание фсин получать генерал-лейтенант четыре генерал-майор генерал-лейтенант евгений устинов командовать войско центральный военный округ становиться генерал-полковник командующий балтий

In [192]:
comparison_result(263,264,columns_lst, columns_final_ex)

id_true_ai-forever-rubert-base id_pred_ai-forever-rubert-base
model: ai-forever-rubert-base
text_lemmatize: улица бутик барвиха luxury village украшать новый год мотив сказка гофман щелкунчик мышиный король это сообщаться пресс-релиз присылать редакция лента ру четверг декабрь центр внимание красавица мари главный героиня сказка фигура главный положительный герой щелкунчик устанавливать праздничный елка возле barvikha hotel spa украшать бутафорский конфета игрушка бант коробка подарок сказочный антогонист семиголовый мышиный король венчать стилизованный торт праздничный карусель карусель оккупировать армия мышиный король охранять золотой орех бронзовый пушка самый романтичный инсталляция па-де-де который танцевать фея драже принц оршад силуэт мягко подсвечивать улица бутик звучать музыка знаменитый балет петр ильич чайковский барабанный бой фанфара дивертисмент вальс цветок заключительный лирический адажио ранее ноябрь московский универмаг цум открывать традиционный рождественский база